# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline

In [23]:
df_raw = pd.read_csv("../../dataset/admissions.csv")
df = df_raw.dropna()
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [24]:
# frequency table for prestige and whether or not someone was admitted
df.prestige.value_counts()

2.0    148
3.0    121
4.0     67
1.0     61
Name: prestige, dtype: int64

In [25]:
df.admit.value_counts()

0    271
1    126
Name: admit, dtype: int64

## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [26]:
dummy_ranks = pd.get_dummies(df.prestige, prefix = 'prestige')

In [27]:
dummy_ranks

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
5,0.0,1.0,0.0,0.0
6,1.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0
8,0.0,0.0,1.0,0.0
9,0.0,1.0,0.0,0.0


#### Question 2.2. When modeling our class variables, how many do we need?

Answer: Since we can figure out the 4th by knowing the other three, we only need three variables for the pretige dummy variables

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [28]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           0.0           1.0           0.0
1      1  660.0  3.67           0.0           0.0           1.0           0.0
2      1  800.0  4.00           1.0           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           0.0           1.0


In [39]:
handCalc.rename(columns={'prestige_1.0': 'prestige_1', 'prestige_2.0': 'prestige_2', 'prestige_3.0': 'prestige_3', 'prestige_4.0': 'prestige_4'}, inplace=True)

In [41]:
# crosstab prestige 1 admission
pd.crosstab(handCalc.admit, handCalc.prestige_1)

prestige_1,0.0,1.0
admit,,
0,243,28
1,93,33


In [46]:
# frequency table cutting prestige and whether or not someone was admitted
# handCalc.prestige_1.value_counts()
pd.crosstab(handCalc.admit, handCalc.prestige_2)

prestige_2,0.0,1.0
admit,,
0,176,95
1,73,53


In [47]:
pd.crosstab(handCalc.admit, handCalc.prestige_3)

prestige_3,0.0,1.0
admit,,
0,178,93
1,98,28


In [48]:
pd.crosstab(handCalc.admit, handCalc.prestige_4)

prestige_4,0.0,1.0
admit,,
0,216,55
1,114,12


In [50]:
pd.crosstab(handCalc.admit, [handCalc.prestige_1, handCalc.prestige_2, handCalc.prestige_3, handCalc.prestige_4], margins=True)

prestige_1 0.0           1.0  All
prestige_2 0.0       1.0 0.0     
prestige_3 0.0  1.0  0.0 0.0     
prestige_4 1.0  0.0  0.0 0.0     
admit                            
0           55   93   95  28  271
1           12   28   53  33  126
All         67  121  148  61  397

#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

In [51]:
# The odds for an event is the ratio of the number of ways the event can occur compared to the number of ways it does not occur.
prob_p1 = 33 / 61.
odds_p1 = 33 / (61. - 33)

print prob_p1
print odds_p1

0.540983606557
1.17857142857


#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [52]:
prob_nop1 = 243 / 336.
odds_nop1 = 243 / (336. - 243)

print prob_nop1
print odds_nop1

0.723214285714
2.61290322581


#### Question 3.3. Calculate the odds ratio.

In [53]:
odds_p1 / odds_nop1

0.4510582010582011

#### Question 3.4. Write this finding in a sentenance:

Answer: For prestige 1 students, the odds of being admitted are 0.45 times than the odds for a prestige 2,3,4 students being admitted.


#### Question 3.5. Print the cross tab for prestige_4.

In [54]:
pd.crosstab(handCalc.admit, handCalc.prestige_4)

prestige_4,0.0,1.0
admit,,
0,216,55
1,114,12


#### Question 3.6. Calculate the OR.

In [55]:
prob_p4 = 12 / 77.
odds_p4 = 12 / (77. - 12)

print prob_p4
print odds_p4

0.155844155844
0.184615384615


In [56]:
prob_nop4 = 114 / 330.
odds_nop4 = 114 / (330. - 114)

print prob_nop4
print odds_nop4

0.345454545455
0.527777777778


#### Question 3.7. Write this finding in a sentence.

In [57]:
odds_p4 / odds_nop4

0.34979757085020247

Answer: For prestige 4 students, the odds of being admitted are 0.35 times than the odds for a prestige 1,2,3 students being admitted.

## Part 4. Analysis

In [58]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           1.0           0.0
1      1  660.0  3.67           0.0           1.0           0.0
2      1  800.0  4.00           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           1.0


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [59]:
# manually add the intercept
data['intercept'] = 1.0

In [60]:
data.rename(columns={'prestige_1.0': 'prestige_1', 'prestige_2.0': 'prestige_2', 'prestige_3.0': 'prestige_3', 'prestige_4.0': 'prestige_4'}, inplace=True)

#### Question 4.1. Set the covariates to a variable called train_cols.

In [66]:
train_cols = data.columns[1:]

logit = sm.Logit(data['admit'], data[train_cols])

#### Question 4.2. Fit the model.

In [67]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### Question 4.3. Print the summary results.

In [68]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 06 Jun 2016   Pseudo R-squ.:                 0.08166
Time:                        23:47:56   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2    -0.6801      0.317     -2.146      0.0

#### Question 4.4. Calculate the odds ratios of the coeffincients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']

In [69]:
print np.exp(result.params)

gre           1.002221
gpa           2.180027
prestige_2    0.506548
prestige_3    0.262192
prestige_4    0.211525
intercept     0.020716
dtype: float64


In [72]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                2.5%     97.5%        OR
gre         1.000074  1.004372  1.002221
gpa         1.136120  4.183113  2.180027
prestige_2  0.272168  0.942767  0.506548
prestige_3  0.133377  0.515419  0.262192
prestige_4  0.093329  0.479411  0.211525
intercept   0.002207  0.194440  0.020716


#### Question 4.5. Interpret the OR of Prestige_2.

Answer: This tells you how a 1 unit increase or decrease in a variable affects the odds of being admitted. We can expect the odds of being admitted to decrease by about 50% if the prestige of a school is 2. 

#### Question 4.6. Interpret the OR of GPA.

Answer: We can expect the odds of being admitted to increase by about 218% for each 1 unit increase in GPA. 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [70]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [71]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### Question 5.1. Recreate the dummy variables.

In [73]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])


#### Question 5.2. Make predictions on the enumerated dataset.

In [75]:
combos['admit_pred'] = result.predict(combos[train_cols])

print combos.tail()

       gre       gpa  prestige  intercept  prestige_2  prestige_3  prestige_4  \
395  800.0  3.806667       4.0        1.0         0.0         0.0         1.0   
396  800.0  4.000000       1.0        1.0         0.0         0.0         0.0   
397  800.0  4.000000       2.0        1.0         1.0         0.0         0.0   
398  800.0  4.000000       3.0        1.0         0.0         1.0         0.0   
399  800.0  4.000000       4.0        1.0         0.0         0.0         1.0   

     admit_pred  
395    0.334286  
396    0.734040  
397    0.582995  
398    0.419833  
399    0.368608  


#### Question 5.3. Interpret findings for the last 4 observations.

Answer: The last four observations all have high GPAs (3.8 and above), as well as identical GRE scores. Admition prediction closely follows the prestige ranking in terms of the prediction.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.